In [1]:
import numpy 
import matplotlib.pyplot as plt
import pandas
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

def load_data(path):
    df = pandas.read_csv(path)
    df = pandas.DataFrame(df)
    return df




if __name__=='__main__':
    #training data below
    #loads data from CSV for PHQ vs. biometric variables

    phqbiodata = load_data(r"rhrphq")
    print(phqbiodata.shape)
    print(list(phqbiodata.columns))
    phqbiodata.head()
    
    #loads data from CSV for GAD vs. biometric variables
    gadbiodata = load_data(r"gadbio")
    print(gadbiodata.shape)
    print(list(gadbiodata.columns))
    gadbiodata.head()
    
    #loads data from CSV for ISI vs. biometric variables
    sleepbiodata = load_data(r"sleepbio")
    print(sleepbiodata.shape)
    print(list(sleepbiodata.columns))
    sleepbiodata.head()
    
    #testdata
#     test_data = load_data(r")

    #fitbit data will always be compared to test data
    
    #start with resting hr, move on to comparing steps, mins active,
    #mins sedentary, mins sleep vs. PHQ scores

    phqstepsx = phqbiodata.iloc[:, 4]
    phqminssleepx= phqbiodata.iloc[:,5]
    phqminssedx = phqbiodata.iloc[:, 6]
    phqexerx = phqbiodata.iloc[:,7]
    phqrhrx = phqbiodata.iloc[:, 8]
    phqscoresy = phqbiodata.iloc[:, 1]
    phqbio_affected = phqbiodata.loc[phqscoresy == 1]
    print(phqbio_affected)
    phqbio_unaffected = phqbiodata.loc[phqscoresy == 0]
    print(phqbio_unaffected)
    
    plt.scatter(phqbio_affected.iloc[:, 0], phqbio_affected.iloc[:, 1], s=10, label ='PHQ/RHR Affected')
    plt.scatter(phqrhr_unaffected.iloc[:, 0], phqrhr_unaffected.iloc[:, 1], s=10, label = 'PHQ/RHR Unaffected')
    plt.legend()
    plt.show()
    
    
    

(14257, 9)
['user_id', 'score', 'user_id.1', 'date', 'steps', 'sleep', 'mins_sedentary', 'mins_exercise', 'resting_hr']
(17412, 8)
['user_id', 'score', 'date', 'steps', 'sleep', 'mins_sedentary', 'mins_exercise', 'resting_hr']
(18683, 8)
['user_id', 'score', 'date', 'steps', 'sleep', 'mins_sedentary', 'mins_exercise', 'resting_hr']
       user_id  score  user_id.1        date  steps  sleep  mins_sedentary  \
45           2      1          2  2018-10-01   4307     30             684   
46           2      1          2  2018-10-02   5518    128             544   
47           2      1          2  2018-10-03   3068    238             594   
48           2      1          2  2018-10-04   4120     76             592   
49           2      1          2  2018-10-05   5618    246             736   
50           2      1          2  2018-10-06   7404    181             533   
51           2      1          2  2018-10-07   5446     95             640   
52           2      1          2  2018-10-

NameError: name 'phqrhr_unaffected' is not defined

In [ ]:
phqbiodata['score'].value_counts()


In [10]:
sns.countplot(x='score', phqbiodata=phqbiodata, palette ='hls')

ValueError: Could not interpret input 'score'

In [ ]:
sns.countplot(x='score', data=phqbiodata, palette='hls')
print(phqbiodata)

plt.show()
plt.savefig('count_plot')

In [ ]:
count_nodepression = len(phqbiodata[phqbiodata['score']==0])
count_depression = len(phqbiodata[phqbiodata['score']==1])
nodep_percent = count_nodepression/(count_depression+count_nodepression)
print('No Depression:',nodep_percent*100)
dep_percent = count_depression/(count_depression+count_nodepression)
print('Depression:', dep_percent*100)


In [14]:
phqbiodata.groupby('score').mean()


,user_id,user_id.1,steps,sleep,mins_sedentary,mins_exercise,resting_hr
score,,,,,,,
0,49.268050,48.922407,9997.527718,479.229710,400.885062,160.842075,74.511120
1,40.298595,40.298595,5048.125963,270.215224,623.894880,19.232895,87.396013


In [ ]:
x = numpy.c_[numpy.ones((phqrhrx.shape[0], 1)), phqrhrx]
y = phqscoresy[:, numpy.newaxis]
theta = numpy.zeros((phqrhrx.shape[1],1))

def sigmoid(x):
    return 1/(1+numpy.exp(-x))

def net_input(theta, x):
    #returns input weighted sums
    return numpy.dot(x, theta)

def probability(theta, x):
    #returns prob with passing of sigmoid
    return sigmoid(net_input(theta, x))
def cost_function(theta, x, y):
    #calculates cost function for training samples
    m = x.shape[0]
    total_cost = -(1/m)* numpy.sum(
    y*numpy.log(probability(theta, x))+ (1-y)*numpy.log(1-probability(theta,x)))
    return total_cost

def gradient(theta, x, y):
    #calcs gradient of cost function at theta
    m = x.shape[0]
    return (1/m)* numpy.dot(x.T, sigmoid(net_input(theta, x))-y)
    
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta, fprime=gradient, args=(x,y.flatten()))
    return opt_weights[0]
parameters = fit(x, y, theta) #model parameters

#plotting decision boundary by setting weighted sum of inputs to 0
x_values = [numpy.min(phqrhrx[:, 1]-5), numpy.max(phqrhrx[:, 2]+5)]
y_values = - (parameters[0] + numpy.dot(parameters[1], x_values))/parameters[2]

plt.plot(x_values, y_values, label = 'Decision Boundary')
plt.xlabel('1st')
plt.ylabel('2nd')
plt.legend()
plt.show()

def predict(self,x):
    theta = parameters[:, numpy.newaxis]
    return probability(theta,x)

def accuracy(self, x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x)>= probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = numpy.mean(predicted_classes == actual_classes)
    return accuracy * 100

accuracy(phqrhrx, y.flatten())

#training model by hand (without sklearn)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()
model.fit(phqrhrx, y)
predicted_classes = model.predict(phqrhrx)
accuracy = accuracy_score(y.flatten(), predicted_classes)
parameters = model.coef_
#using sklearn logistic regression model

ModuleNotFoundError: No module named 'sklearn'